In [50]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
import os

In [51]:
train = pd.read_csv('./tensorflow-great-barrier-reef/train-5folds.csv')
train['pos'] = train.annotations != '[]'
fold = 1

In [52]:
HOME_DIR = "./yolo_data_subseq_empty"
DATASET_PATH = os.path.join(f'fold{str(fold)}', 'images')
LABELS_PATH = os.path.join(f'fold{str(fold)}', 'labels')

os.makedirs(os.path.join(HOME_DIR, f'fold{str(fold)}'))

for path in [DATASET_PATH, LABELS_PATH]:
    os.makedirs(os.path.join(HOME_DIR, path))
    os.makedirs(os.path.join(HOME_DIR, path, 'train'))
    os.makedirs(os.path.join(HOME_DIR, path, 'val'))

In [53]:
# sharp_filter = np.array([[0,-1,0], [-1,5,-1], [0,-1,0]])
# def gamma_enhance(img, gamma=1.0):
#     lookUpTable = np.empty((1,256), np.uint8)
#     for i in range(256):
#         lookUpTable[0,i] = np.clip(pow(i / 255.0, gamma) * 255.0, 0, 255)
#     return cv2.LUT(img, lookUpTable)

In [54]:
# # Preprocessed Images

# for i in tqdm(range(len(train))):
#     row = train.loc[i]
#     img = cv2.imread(row.image_path)
#     # img = gamma_enhance(cv2.filter2D(img, -1, sharp_filter), gamma=2)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     if row.video_id != fold:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/train/{row.image_id}.jpg')
#     else:
#         cv2.imwrite(f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg', img)
#         # copyfile(f'{row.image_path}', f'{HOME_DIR}{DATASET_PATH}/val/{row.image_id}.jpg') 

In [55]:
train_df = train[train['fold']!=fold]
temp = train_df[train_df['pos'] == False].groupby('video_id').sample(400//3, random_state=1).index
train.loc[train.index[temp], 'pos'] = True

In [56]:
train[(train['fold'] != 1) & (train['pos'] == True)]

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,n_annotations,has_annotations,image_path,subsequence_id,fold,pos
4,0,40258,4,4,0-4,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,True
6,0,40258,6,6,0-6,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,1,2,True
16,0,40258,16,16,0-16,"[{'x': 559, 'y': 213, 'width': 50, 'height': 32}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,2,4,True
17,0,40258,17,17,0-17,"[{'x': 558, 'y': 213, 'width': 50, 'height': 32}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,2,4,True
18,0,40258,18,18,0-18,"[{'x': 557, 'y': 213, 'width': 50, 'height': 32}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,2,4,True
...,...,...,...,...,...,...,...,...,...,...,...,...
23371,2,29859,10630,2858,2-10630,"[{'x': 65, 'y': 606, 'width': 41, 'height': 37}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,136,4,True
23372,2,29859,10631,2859,2-10631,"[{'x': 51, 'y': 643, 'width': 44, 'height': 37}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,136,4,True
23373,2,29859,10632,2860,2-10632,"[{'x': 38, 'y': 681, 'width': 46, 'height': 37}]",1,True,../input/tensorflow-great-barrier-reef/train_i...,136,4,True
23408,2,29859,10667,2895,2-10667,[],0,False,../input/tensorflow-great-barrier-reef/train_i...,137,3,True


In [57]:
annos = []
for i, x in train.iterrows():
    if x.fold == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'./tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'{HOME_DIR}/fold{fold}/images/{mode}/{x.image_id}.jpg')
    r = ''
    anno = eval(x.annotations)
    if not x.pos or len(anno) == 0:
        continue
    for an in anno:
#            annos.append(an)
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
    with open(f'{HOME_DIR}/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)